In [1]:
from phasic_tonic.detect_phasic import detect_phasic, detect_phasic_v2
from phasic_tonic.PhasicTonic import PhasicTonic
from phasic_tonic.DatasetLoader import DatasetLoader
from phasic_tonic.helper import get_metadata
from phasic_tonic.runtime_logger import logger_setup
from phasic_tonic.utils import get_sequences, get_segments, load_data, get_start_end, preprocess

import math
import numpy as np
import pandas as pd
import pynapple as nap

from pathlib import Path
from tqdm.auto import tqdm
from scipy.io import loadmat

fs = 500
fs_cbd = 2500
fs_os = 2500
fs_rgs = 1000

targetFs = 500
n_down_cbd = fs_cbd/targetFs
n_down_rgs = fs_rgs/targetFs
n_down_os = fs_os/targetFs

logger = logger_setup()

CONFIG_DIR = "/home/nero/phasic_tonic/data/dataset_loading.yaml"
DATASET_DIR = "/home/nero/datasets/preprocessed"
OUTPUT_DIR1 = "/home/nero/phasic_tonic/data/analysis_output/whole_posttrial5/"
OUTPUT_DIR2 = "/home/nero/phasic_tonic/data/analysis_output/segmented_posttrial5/"

def half_round_up(n):
    if n - math.floor(n) < 0.5:
        return math.floor(n)
    return math.ceil(n)

compressed_datasets = list(Path(DATASET_DIR).glob('*.npz'))

Datasets = DatasetLoader(CONFIG_DIR)
mapped_datasets = Datasets.load_datasets()

len(compressed_datasets)

699

# Whole post-trial 5

In [2]:
container = []

with tqdm(mapped_datasets) as mapped_tqdm:
    for name in mapped_tqdm:
        metadata = get_metadata(name)
        mapped_tqdm.set_postfix_str(name)
        states_fname, hpc_fname, _ = mapped_datasets[name]
        logger.debug("Loading: {0}".format(name))

        if metadata["treatment"] == 0 or metadata["treatment"] == 1:
            n_down = n_down_cbd
        elif metadata["treatment"] == 2 or metadata["treatment"] == 3:
            n_down = n_down_rgs
        elif metadata["treatment"] == 4:
            n_down = n_down_os
        
        # Load the LFP data
        lfpHPC = loadmat(hpc_fname)['HPC'].flatten()
        # Load the sleep states
        hypno = loadmat(states_fname)['states'].flatten()
        
        # Skip if no REM epoch is detected
        if(not (np.any(hypno == 5))):
            logger.debug("No REM detected. Skipping.")
            continue
        elif(np.sum(np.diff(get_sequences(np.where(hypno == 5)[0]))) < 10):
            logger.debug("No REM longer than 10s. Skipping.")
            continue
        
        # Detect phasic intervals
        lfpHPC_down = preprocess(lfpHPC, n_down)
        phREM = detect_phasic(lfpHPC_down, hypno, targetFs)
        break
        # Classify each time window as tonic or phasic event
        for rem_idx in phREM:
            # Container for storing results and metadata
            data = metadata.copy()
            
            rem_start, rem_end = rem_idx[0], rem_idx[1]
            data["rem_start"] = rem_start
            data["rem_end"] = rem_end

            phasic_idx = phREM[rem_idx]

            # Fill the whole REM epoch as Tonic state
            result = ["T" for _ in range(rem_end-rem_start)] 

            # print(f"REM epoch {rem_idx}")
            if phasic_idx:
                for ph in phasic_idx:
                    # Round up the phasic timestamp if its fractional part is greater than 0.5
                    ph_start, ph_end = half_round_up(ph[0]/500), half_round_up(ph[1]/500)

                    # print(f"({ph[0]/500}, {ph[1]/500}) -> ({ph_start}, {ph_end})")
                    ph_start -= rem_start
                    ph_end -= rem_start
                    
                    # print(ph_start, ph_end)
                    result[ph_start:ph_end] = "P"
            
            # print("".join(result))
            data["event_strings"] = "".join(result)
            container.append(data)
df = pd.DataFrame(container)

  0%|          | 0/539 [00:00<?, ?it/s]

In [3]:
phREM

{(1331, 1485): [(689992, 690885), (706502, 707232), (729634, 730948)],
 (2428, 2475): [(1214577, 1216154)]}

In [ ]:
for rem_idx in phREM:
    # Container for storing results and metadata
    data = metadata.copy()
    
    rem_start, rem_end = rem_idx[0], rem_idx[1]
    data["rem_start"] = rem_start
    data["rem_end"] = rem_end

    phasic_idx = phREM[rem_idx]


In [ ]:
for rem_idx in phREM:
    # Container for storing results and metadata
    data = metadata.copy()
    
    rem_start, rem_end = rem_idx[0], rem_idx[1]
    data["rem_start"] = rem_start
    data["rem_end"] = rem_end

    phasic_idx = phREM[rem_idx]

    # Fill the whole REM epoch as Tonic state
    result = ["T" for _ in range(rem_end-rem_start)] 

    # print(f"REM epoch {rem_idx}")
    if phasic_idx:
        for ph in phasic_idx:
            # Round up the phasic timestamp if its fractional part is greater than 0.5
            ph_start, ph_end = half_round_up(ph[0]/500), half_round_up(ph[1]/500)

            # print(f"({ph[0]/500}, {ph[1]/500}) -> ({ph_start}, {ph_end})")
            ph_start -= rem_start
            ph_end -= rem_start
            
            # print(ph_start, ph_end)
            result[ph_start:ph_end] = "P"
    
    # print("".join(result))
    data["event_strings"] = "".join(result)
    container.append(data)
